<a href="https://colab.research.google.com/github/Satya-Prakash009/EDA-playstore-project/blob/main/EDA_Project_Play_Store_App_Review_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b> The Play Store apps data has enormous potential to drive app-making businesses to success. Actionable insights can be drawn for developers to work on and capture the Android market. </b>

## <b> Each app (row) has values for catergory, rating, size, and more. Another dataset contains customer reviews of the android apps.</b>

## <b> Explore and analyze the data to discover key factors responsible for app engagement and success. </b>

1. Google Play Store apps and reviews
Mobile apps are everywhere. They are easy to create and can be lucrative. Because of these two factors, more and more apps are being developed. In this notebook, we will do a comprehensive analysis of the Android app market by comparing over ten thousand apps in Google Play across different categories. We'll look for insights in the data to devise strategies to drive growth and retention.

Let's take a look at the data, which consists of two files:

apps.csv: contains all the details of the applications on Google Play. There are 13 features that describe a given app.
user_reviews.csv: contains 100 reviews for each app, most helpful first. The text in each review has been pre-processed and attributed with three new features: Sentiment (Positive, Negative or Neutral), Sentiment Polarity and Sentiment Subjectivity.

In [ ]:
# import package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline
sns.set(color_codes=True)
 
path = '/content/drive/MyDrive/Colab Notebooks/Data for class/PlayStoreData.csv'
play_df = pd.read_csv(path)
play_df.info()

In [ ]:
play_df.head()

In [ ]:
play_df.tail()

In [ ]:
play_df.dtypes

**2. Data cleaning**
The three features that we will be working with most frequently henceforth are Installs, Size, and Price. A careful glance of the dataset reveals that some of these columns mandate data cleaning in order to be consumed by code we'll write later. Specifically, the presence of special characters (, $ +) and letters (M k) in the Installs, Size, and Price columns make their conversion to a numerical data type difficult. Let's clean by removing these and converting each column to a numeric type.

In [84]:
# if rating column has Nan values, it is replaced by mean value
play_df.fillna(value=play_df['Rating'].median(), inplace=True)

varies with device ko category ka median value of category... code

In [61]:
# if size column has string values, that row is to be deleted

play_df = play_df[play_df["Size"].str.contains("Varies with device")==False]

In [ ]:
# if Installs column has string values, that row is to be deleted

play_df= play_df[play_df["Installs"].str.contains("Free")==False]
print(play_df)

In [ ]:
# every 'k' in size to be should be removed and "*0.001" needs to be done
for play_df["Size"].str.contains("k"):
  play_df["Size"] =*0.001

In [ ]:
# REMOVE ALL SPECIAL CHARACTERS 
chars_to_remove = ['+', ',', 'M', '$']
cols_to_clean = ['Installs', 'Size', 'Price']

# Loop for each column
for col in cols_to_clean:
    # Replace the characters in each column col with an empty string
    for char in chars_to_remove:
        play_df[col] = play_df[col].replace(char, ' ')
     
play_df.head(10)

In [ ]:
# Convert col to numeric
for col in cols_to_clean:
  play_df[col] = pd.to_numeric(play_df[col])

In [ ]:
# if genre more than one,only one to be kept, rest discarded


In [ ]:
# all "and up" from version to be removed